## Step 0: Importing the Appropriate Packages

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
import astropy
import emcee
import corner
import os
import glob

import astropy.constants as ac
from astropy import units as u
from astropy.convolution import Gaussian1DKernel, convolve
from astropy.modeling import models
from astropy.visualization import quantity_support
from scipy.optimize import minimize
from scipy import interpolate

## Step 1: Importing Data and Changing the Units of the Variables

We'll first import and read the following data:

1. chen+07_slshlh.dat; This contains the Spitzer IRS spectrum of BetaPic from ~5-35 microns. 

2. betapic_fluxes.dat; This contains assorted photometry from the optical and infrared with BetaPic. 

3. betapic_lws_cnt3_.txt; This contains the ISO LWS spectrum of BetaPic, from ~43 to 160 microns.

### 1.1: Using the correct folders to pull the data from and to put the data in

In [2]:
# first the main directory
cwd = os.getcwd()

# then the appropriate folders inside this main directory
Photom_IRS = cwd + "\\data\\betapic_data\DiskModels\Debris\BetaPic" # where the photometry and IRS Spectrum is found
LWS = cwd + "\\data\\betapic_data\DiskModels\Debris\BetaPic\iso\hpdp_62003530_3" # where the LWS Spectrum is found

Ice = cwd + "\\data\\water_ice_opacities_lsbrp" # where the different opacity files are found
Silicates = cwd + "\\data\\silicates_for_betapic" # where the different silicate files are found

### 1.2: Importing the Data 

In [3]:
# Changing directory to the photometry and IRS Spectrum files
os.chdir(Photom_IRS)

# Columns are all labelled (wavelength in microns, flux (F_nu) and uncertainty in Janskys, (Jy)).
Spitzer_IRS = np.loadtxt('chen+07_slshlh.dat')
Wavelength_IRS, Flux_IRS_Jy, Error_IRS_Jy = Spitzer_IRS[:,0], Spitzer_IRS[:,1], Spitzer_IRS[:,2]

# Columns are formatted as wavelength (mircons), flux (F_nu, Jy), uncertainty (Jy).
Photom = np.loadtxt('betapic_fluxes.dat')
Wavelength_Photom, Flux_Photom_Jy, Error_Photom_Jy = Photom[:,0], Photom[:,1], Photom[:,2]


# Changing directory to the LWS Spectrum file
os.chdir(LWS)

# Columns are formatted  as wavelength (microns), nu*F_nu (erg/cm^2/s), and uncertainty in nu*F_nu (same units).
ISO_LWS = np.loadtxt('betapic_lws_cnt3_.txt')
Wavelength_LWS, Flux_LWS, Error_LWS = ISO_LWS[:,0], ISO_LWS[:,1], ISO_LWS[:,2]

### 1.3: Rewriting the units
We need to convert the fluxes and error from Jansky to erg cm$^{-2}$ s$^{-1}$ . We know that 1 Jy equals to 10$^{23}$ erg cm$^{-2}$ s$^{-1}$ Hz$^{-1}$.

Such that we get: erg cm$^{-2}$ s$^{-1}$ = 10$^{-23}$ Hz Jy

In [4]:
# We can now calculate the flux using F = v*F_v (also making sure that the wavelength is in meters and not in microns)
Flux_IRS = 10**(-23)*Flux_IRS_Jy*(ac.c.value/(Wavelength_IRS*10**(-6)))
Flux_Photom = 10**(-23)*Flux_Photom_Jy*(ac.c.value/(Wavelength_Photom*10**(-6)))

# We can now also calculate the error in the fluxes
Error_IRS = 10**(-23)*Error_IRS_Jy*(ac.c.value/(Wavelength_IRS*10**(-6)))
Error_Photom = 10**(-23)*Error_Photom_Jy*(ac.c.value/(Wavelength_Photom*10**(-6)))

In [5]:
# Making sure no flux is zero/negative
Flux_IRS, Wavelength_IRS, Error_IRS = Flux_IRS[Flux_IRS>0], Wavelength_IRS[Flux_IRS>0], Error_IRS[Flux_IRS>0]
Flux_Photom, Wavelength_Photom, Error_Photom = Flux_Photom[Flux_Photom>0], Wavelength_Photom[Flux_Photom>0], Error_Photom[Flux_Photom>0]

# Removing the negative values
Flux_LWS, Wavelength_LWS, Error_LWS = Flux_LWS[Flux_LWS>0], Wavelength_LWS[Flux_LWS>0], Error_LWS[Flux_LWS>0]

# Removing the error equal to 0 from the photometry
Error_Photom, Flux_Photom, Wavelength_Photom = Error_Photom[Error_Photom!=0], Flux_Photom[Error_Photom!=0], Wavelength_Photom[Error_Photom!=0]

We need to convolve LWS Spectrum to smoothen it

In [6]:
# Create kernel & convolving lWS spectrum
g = Gaussian1DKernel(stddev=3)
z = convolve(Flux_LWS, g)

# Removing the tail of the LWS spectrum [by trial and error]
Wavelength_z, z, Error_z = Wavelength_LWS[6:], z[6:], Error_LWS[6:]

In [7]:
# Creating a list with all ice opacity file names
os.chdir(Ice)
ice = sorted(glob.glob("h2oice.p3p5.amin0p005.amax*.wb08c146"))

# Columns have the format: wavelength (microns), total opacity, albedo, absorption coefficient, asymmetry parameter g
# Read the files in with np.loadtxt('filename')

# Creating a list with all silicate opacity file names
os.chdir(Silicates)
silicates = sorted(glob.glob("sil.p3p5.amax*.extinc"))

# Returning to main directory
os.chdir(cwd)

## Chi Squared Fit

### Reading in the Ice and Silicate Files

In [8]:
def tau(kappa, mass, d):
    return kappa*(mass*(ac.M_earth).value*10**(3))/(d**2)

def chi_sq(model, flux, error):
    """Calculates the chi squared value."""
    return np.sum(((model-flux)/error)**2)

def spectrum_ice_si(wave, x_b, x_d,  kappa_ice, kappa_si, d):
    """Creates a model for the flux of a star, for which a blackbody approximation is used,
    in combination with a ring for which both ice and silicates opacities are
    taken into account."""
    T, SolidAngle = x_b
    T_2, mass_i, mass_s = x_d
    
    bb = models.BlackBody(temperature=T*u.K)
    B_q_1 = bb(wave*u.micron)
    bb_2 = models.BlackBody(temperature=T_2*u.K)
    B_q_2 = bb_2(wave*u.micron)
    
    t_ice = tau(kappa_ice, mass_i, d)/0.0005
    t_si = tau(kappa_si, mass_s, d)/0.004
        
    F_1 = (SolidAngle)*B_q_1*(ac.c/(wave*u.micron))
    F_2 = B_q_2*(ac.c/(wave*u.micron))
    
    F = F_1 + (t_ice + t_si)*F_2
    return F.to(u.erg * u.cm**(-2) * u.s**(-1) * u.sr**(-1)).value

def chi_squared_ice_si(x0, wave, x0_beta, flux_ISO, error_ISO, d, kappa_ice, kappa_si):
    """Calculates the reduced chi^2 value for a given set of parameters and data points
    (with the corresponding errors) for a certain model. This definition is for the 
    the determination of the reduced chi^2 value of the flux of a model."""
    for i in x0:
        if i < 0:
            return np.inf
    
    model = spectrum_ice_si(wave, x0_beta, x0, kappa_ice, kappa_si, d)
    chi = chi_sq(model, flux_ISO, error_ISO)
    return chi/(len(model))

In [9]:
phi = [80, 1e-5, 1e-2] # phi is [dust temperature, mass ice, mass silicates]

fit_mask = ( (Wavelength_z<=100) )

d_BetaPic = (19.76*u.pc).to(u.cm).value
x0_BetaPic = [8200, 1.2699*10**(-17)]

estimates_LWS=[]
chi_LWS = []
array_chi_LWS_ = np.array([[0,0,0,0,0,0]])

silicates_best = []
estimates_j = []

for i in range(len(ice)): # Creating a for loop to loop over all ice opacity files
    chi_j = []
    
    os.chdir(Ice)
    Ice_i = np.loadtxt(ice[i],skiprows=1)
    
    f_i = interpolate.interp1d(Ice_i[:,0], Ice_i[:,1]) #Interpolating to make the array's match 
    kappa_ice_new = f_i(Wavelength_z[fit_mask])

    for j in range(len(silicates)): # Creating a for loop to loop over all silicate files
        os.chdir(Silicates)
        silicate_j = np.loadtxt(silicates[j],skiprows=1)
        
        f_si = interpolate.interp1d(silicate_j[:,0], silicate_j[:,1]) #Interpolating to make the array's match 
        kappa_si_new = f_si(Wavelength_z[fit_mask])
        
        estimates_i = minimize(fun=chi_squared_ice_si, x0 = phi, args=(Wavelength_z[fit_mask], x0_BetaPic, 
                                                                   z[fit_mask], Error_z[fit_mask], 
                                                                   d_BetaPic, kappa_ice_new, kappa_si_new), 
                           method = 'Nelder-Mead')

        chi_j.append(estimates_i.fun)
        estimates_j.append(estimates_i.x)
        
        # Such that we can see all possible combinations with their final chi-value (+variables)
        array_chi_LWS_ = np.append(array_chi_LWS_, [[ice[i], silicates[j], estimates_i.fun, estimates_i.x[0],
                                                   estimates_i.x[1], estimates_i.x[2]]], axis = 0)

        
    j_best = np.where(chi_j==np.min(chi_j))[0][0] # Finding which silicate file is best for the i'th ice opacity file
    silicates_best.append(j_best)
    
    chi_LWS.append(chi_j[j_best])
    estimates_LWS.append(estimates_j[j_best])

best_i_LWS = np.where(chi_LWS==np.min(chi_LWS))[0][0] # Finding which ice file is best for the fit

print('index ice:', best_i_LWS, " - ", ice[best_i_LWS])
print('index silicates:', silicates_best[best_i_LWS], " - ", silicates[silicates_best[best_i_LWS]])
print('chi^2:', chi_LWS[best_i_LWS])
print('[T_1, M_I, M_S1]:', estimates_LWS[best_i_LWS])

index ice: 2  -  h2oice.p3p5.amin0p005.amax15.g.ab0p0005.wb08c146
index silicates: 5  -  sil.p3p5.amax1mm.g.ab0p004.extinc
chi^2: 0.09119760765264488
[T_1, M_I, M_S1]: [7.83099651e+01 1.96042592e-04 1.34327379e-02]
